In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [3]:
### load the dataset

data = pd.read_csv('Churn_Modelling.csv')

In [4]:
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
data.shape

(10000, 14)

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


Preprocessing of the Data
  
  Drop irrelevant columns

In [7]:
data=data.drop(['RowNumber','CustomerId','Surname'], axis=1)



In [8]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [9]:
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])

In [10]:
data['Gender'].value_counts()

Gender
1    5457
0    4543
Name: count, dtype: int64

In [11]:
data['Geography'].value_counts()

Geography
France     5014
Germany    2509
Spain      2477
Name: count, dtype: int64

In [12]:
from sklearn.preprocessing import OneHotEncoder
geo_oh_encoder = OneHotEncoder()
geo_encoder = geo_oh_encoder.fit_transform(data[['Geography']])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [13]:
geo_oh_encoder.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [14]:
geo_encoder_df = pd.DataFrame(geo_encoder.toarray(),columns=geo_oh_encoder.get_feature_names_out(['Geography']))

In [15]:
geo_encoder_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [16]:
data = pd.concat([data.drop('Geography',axis=1),geo_encoder_df],axis=1)

In [17]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [18]:
### save the encoders and scalers

with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(geo_oh_encoder,file)

In [19]:
### divide the data into independent and dependent features
X = data.drop('Exited',axis=1)
y = data['Exited']


In [20]:
X_train,X_test,y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42) 

### scaling the features

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [21]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)


In [22]:
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [42]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [43]:
(X_train.shape[1],)

(12,)

In [44]:
### Build ANN Model

model = Sequential(
    [Dense(64,activation = 'relu',input_shape=(X_train.shape[1],1)), ##First HL connected with input layer
     Dense(32,activation='relu'),##HL2
    Dense(1,activation='sigmoid') , ##Output layer (binary calssification, that's why used Sigmoid activation function)


    ]



)

In [45]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 12, 64)            128       
                                                                 
 dense_4 (Dense)             (None, 12, 32)            2080      
                                                                 
 dense_5 (Dense)             (None, 12, 1)             33        
                                                                 
Total params: 2241 (8.75 KB)
Trainable params: 2241 (8.75 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [60]:
import tensorflow
opt = tensorflow.keras.optimizers.Adam(learning_rate =0.01)
loss = tensorflow.keras.losses.BinaryCrossentropy()
loss

In [59]:
### compile the model

model.compile(optimizer=opt,loss='BinaryCrossentropy',metrics=['accuracy'])

In [61]:
### setup the Tensorboard
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
log_dir = "logs/fit/" + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
tensorflow_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)

In [62]:
### set up early stopping
early_stopping_callback = EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [64]:
#Training The Model

history = model.fit( 
    X_train,y_train, validation_data=(X_test,y_test),epochs=100, callbacks=[tensorflow_callback,early_stopping_callback]
)


Epoch 1/100
250/250 [==============================] - 1s 4ms/step - loss: 0.5047 - accuracy: 0.7945 - val_loss: 0.4913 - val_accuracy: 0.8037
Epoch 2/100
250/250 [==============================] - 1s 4ms/step - loss: 0.5047 - accuracy: 0.7945 - val_loss: 0.4920 - val_accuracy: 0.8036
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 0.5043 - accuracy: 0.7945 - val_loss: 0.4952 - val_accuracy: 0.8039
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 0.5048 - accuracy: 0.7944 - val_loss: 0.4912 - val_accuracy: 0.8036
Epoch 5/100
250/250 [==============================] - 1s 4ms/step - loss: 0.5045 - accuracy: 0.7945 - val_loss: 0.4910 - val_accuracy: 0.8040
Epoch 6/100
250/250 [==============================] - 1s 5ms/step - loss: 0.5053 - accuracy: 0.7945 - val_loss: 0.4920 - val_accuracy: 0.8036
Epoch 7/100
250/250 [==============================] - 1s 6ms/step - loss: 0.5045 - accuracy: 0.7945 - val_loss: 0.4912 - val_accuracy: 0.8036

In [68]:
model.save('model.h5')

In [69]:
## Load tenserboard extension

%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [71]:
%tensorboard --logdir=logs/fit/20240929-203449

In [54]:
### Load The Pickle File
